In [1]:
from agent.const import (
    ChatConfig,
    Text2SQLConfig,
    GEMINI_FAST_CONFIG,
    GPT4O_MINI_CONFIG,
    GPT4O_CONFIG,
    TEXT2SQL_FASTEST_CONFIG,
    TEXT2SQL_FAST_OPENAI_CONFIG,
    TEXT2SQL_DEEPSEEK_V3_CONFIG,
    TEXT2SQL_DEEPSEEK_V3_FAST_CONFIG
)

from agent.prompt.prompt_controller import (
    PromptConfig, 
    VERTICAL_PROMPT_BASE, 
    VERTICAL_PROMPT_UNIVERSAL,
    HORIZONTAL_PROMPT_BASE,
    HORIZONTAL_PROMPT_UNIVERSAL,
    FIIN_VERTICAL_PROMPT_UNIVERSAL,
)

from initialize import initialize_text2sql


/opt/miniconda3/envs/text2sql/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-26 21:06:00,404 - INFO - PyTorch version 2.5.1 available.


In [2]:
import os

import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


In [3]:
text2sql_config = TEXT2SQL_DEEPSEEK_V3_CONFIG
prompt_config = VERTICAL_PROMPT_UNIVERSAL

In [4]:
text2sql = initialize_text2sql(text2sql_config, prompt_config)

2025-01-26 21:06:02,105 - INFO - Checking embedding server at http://localhost:8080/embed
2025-01-26 21:06:02,188 - INFO - Response embedding server: <Response [200]>
2025-01-26 21:06:02,188 - INFO - Using remote embedding server
2025-01-26 21:06:02,188 - INFO - Finish setup embedding
2025-01-26 21:06:02,189 - INFO - Checking embedding server at http://localhost:8081
2025-01-26 21:06:02,442 - INFO - Response embedding server: <Response [200]>
2025-01-26 21:06:02,444 - INFO - Found reranking server at: http://localhost:8081
2025-01-26 21:06:02,445 - INFO - Finish setup reranker, using reranker api
2025-01-26 21:06:02,462 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-01-26 21:06:02,552 - INFO - Finish setup db
2025-01-26 21:06:02,614 - INFO - Finish setup text2sql


postgres
Using OpenAI Wrapper with host https://api.deepseek.com
Using OpenAI Wrapper with host https://api.deepseek.com


In [13]:
question = "For the year 2022, what was the Return on Equity (ROE) for Vietcombank (VCB) and Techcombank (TCB)?"


In [14]:
his, err, table = text2sql.solve(question)

2025-01-26 21:11:41,713 - INFO - Get stock code based on company name response
2025-01-26 21:11:42,094 - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-01-26 21:11:44,835 - INFO - Completion time of deepseek-chat: 3.121119976043701s


CompletionUsage(completion_tokens=49, prompt_tokens=179, total_tokens=228, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=64), prompt_cache_hit_tokens=64, prompt_cache_miss_tokens=115)


2025-01-26 21:11:45,642 - INFO - Time taken to find stock code similarity: 0.8059403896331787
2025-01-26 21:11:45,643 - INFO - Connecting to database postgres, postgres...
2025-01-26 21:11:46,474 - INFO - Connecting to database postgres, postgres...
2025-01-26 21:11:46,487 - INFO - Time taken to return mapping table: 0.8181347846984863


['ROLTOA', 'ROE', 'ROI', 'ROIC']
['ROLTOA', 'ROE', 'ROI', 'ROIC']
[]


2025-01-26 21:11:47,626 - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-01-26 21:12:00,289 - INFO - Completion time of deepseek-chat: 13.014074802398682s
2025-01-26 21:12:00,291 - INFO - Connecting to database postgres, postgres...
2025-01-26 21:12:00,336 - INFO - Time taken: 18.622678995132446s


CompletionUsage(completion_tokens=495, prompt_tokens=2860, total_tokens=3355, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=2176), prompt_cache_hit_tokens=2176, prompt_cache_miss_tokens=684)
stock_code
company_name
en_company_name
industry
is_bank
is_securities
ratio_code
ratio_name


In [15]:
for tab in table:
    print(tab)
    print(tab.table)

Table(desc = Company Info)
  stock_code                                       company_name  \
0        VCB  Ngân hàng Thương mại Cổ phần Ngoại thương Việt...   
1        TCB    Ngân hàng Thương mại Cổ phần Kỹ thương Việt Nam   

                                     en_company_name industry  
0  Joint Stock Commercial Bank For Foreign Trade ...  Banking  
1  Vietnam Technological And Commercial Joint Sto...  Banking  
Table(desc = category_code_ratio)
  ratio_code        ratio_name
2        ROE  Return On Equity
Table(desc =  SQL 1 Result: ROE for Vietcombank (VCB) and Techcombank (TCB) in 2022 (Annual Report))
  stock_code  year  quarter category_code      data
0        TCB  2022        0           ROE  0.180176
1        VCB  2022        0           ROE  0.220567


In [17]:
for message in his:
    role = message['role']
    content = message['content']

    print(f"Role: {role}")
    print('=============================')
    print(content)
    print('=============================')

Role: system

    You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query. If time not mentioned, assume collecting data in Q3 2024.
    You will have the following database description:

    ### Database Description

    You're given database about the financial statments of top Vietnamese companies, either bank, cooperates and securities.

<overall_description>

All the data in the financial statments are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translation of the 
categories are followed by the International Financial Reporting Standards (IFRS).

There are 3 type of financial statments, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).
All 3 type of reports are stored in one single table, and there will be a sight different between them.

The database includes two reporting periods 

### Reset Text2SQL

In [7]:
text2sql.reset()

Role: system

    You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query. If time not mentioned, assume collecting data in Q3 2024.
    You will have the following database description:

    ### Database Description

    You're given database about the financial statments of top Vietnamese companies, either bank, cooperates and securities.

<overall_description>

All the data in the financial statments are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translation of the 
categories are followed by the International Financial Reporting Standards (IFRS).

There are 3 type of financial statments, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).
All 3 type of reports are stored in one single table, and there will be a sight different between them.

The database includes two reporting periods 

In [ ]:
# from pymilvus import connections, utility

# # Step 1: Connect to the Milvus server
# connections.connect(
#     alias="default",  # Default connection alias
#     host="localhost", # Milvus server host
#     port="19530"      # Milvus server port
# )

# # Step 2: Specify the name of the collection to delete
# collection_names = [
#     'company_name_chroma',
#     'category_bank_chroma',
#     'category_non_bank_chroma',
#     'category_sec_chroma',
#     'category_ratio_chroma',
#     'sql_query',
#     'sql_query_universal',
#     'category_universal_chroma'
# ]

# for collection_name in collection_names:

#     # Step 3: Check if the collection exists
#     if utility.has_collection(collection_name):
#         print(f"Collection '{collection_name}' exists. Deleting it...")
        
#         # Step 4: Drop the collection
#         utility.drop_collection(collection_name)
#         print(f"Collection '{collection_name}' deleted successfully.")
#     else:
#         print(f"Collection '{collection_name}' does not exist.")

Collection 'company_name_chroma' exists. Deleting it...
Collection 'company_name_chroma' deleted successfully.
Collection 'category_bank_chroma' exists. Deleting it...
Collection 'category_bank_chroma' deleted successfully.
Collection 'category_non_bank_chroma' exists. Deleting it...
Collection 'category_non_bank_chroma' deleted successfully.
Collection 'category_sec_chroma' exists. Deleting it...
Collection 'category_sec_chroma' deleted successfully.
Collection 'category_ratio_chroma' exists. Deleting it...
Collection 'category_ratio_chroma' deleted successfully.
Collection 'sql_query' exists. Deleting it...
Collection 'sql_query' deleted successfully.
Collection 'sql_query_universal' exists. Deleting it...
Collection 'sql_query_universal' deleted successfully.
Collection 'category_universal_chroma' exists. Deleting it...
Collection 'category_universal_chroma' deleted successfully.
